# T81-558: Applications of Deep Neural Networks
**Module 13: Advanced/Other Topics**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 13 Video Material

* **Part 13.1: Flask and Deep Learning Web Services** [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)
* Part 13.2: Deploying a Model to AWS  [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)
* Part 13.3: Using a Keras Deep Neural Network with a Web Application  [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)
* Part 13.4: When to Retrain Your Neural Network [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)
* Part 13.5: AI at the Edge: Using Keras on a Mobile Device  [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)


# Part 13.1: Flask and Deep Learning Web Services

* [Flask Quickstart](https://flask.palletsprojects.com/en/1.0.x/quickstart/)

### Flask Hello World

In [ ]:
from werkzeug.wrappers import Request, Response
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

### MPG Flask

Sample JSON

```
{
  "cylinders": 8, 
  "displacement": 300,
  "horsepower": 78, 
  "weight": 3500,
  "acceleration": 20, 
  "year": 76,
  "origin": 1
}
```

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

In [ ]:
pred = model.predict(x_test)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print(f"After load score (RMSE): {score}")

In [ ]:
model.save(os.path.join("./dnn/","mpg_model.h5"))

In [ ]:
cols = [x for x in df.columns if x not in ('mpg','name')]

print("{")
for i,name in enumerate(cols):
    print(f'"{name}":{{"min":{df[name].min()},"max":{df[name].max()}}}{"," if i<(len(cols)-1) else ""}')
print("}")

In [ ]:
import os
from tensorflow.keras.models import load_model
import numpy as np

model = load_model(os.path.join("./dnn/","mpg_model.h5"))
x = np.zeros( (1,7) )

x[0,0] = 8 # 'cylinders', 
x[0,1] = 400 # 'displacement', 
x[0,2] = 80 # 'horsepower', 
x[0,3] = 2000 # 'weight',
x[0,4] = 19 # 'acceleration', 
x[0,5] = 72 # 'year', 
x[0,6] = 1 # 'origin'


pred = model.predict(x)
float(pred[0])

### Flask MPG Client

In [ ]:
import requests

json = {
  "cylinders": 8, 
  "displacement": 300,
  "horsepower": 78, 
  "weight": 3500,
  "acceleration": 20, 
  "year": 76,
  "origin": 1
}

r = requests.post("http://localhost:5000/api/mpg",json=json)
if r.status_code == 200:
    print("Success: {}".format(r.text))
else: print("Failure: {}".format(r.text))

In [26]:
import requests
response = requests.post('http://localhost:5000/api/image', files=dict(image=('hickory.jpeg',open('./photos/hickory.jpeg','rb'))))
if response.status_code == 200:
    print("Success: {}".format(response.text))
else: print("Failure: {}".format(response.text))

Success: {
  "pred": [
    {
      "name": "boxer", 
      "prob": 0.9178280830383301
    }, 
    {
      "name": "American_Staffordshire_terrier", 
      "prob": 0.044582076370716095
    }, 
    {
      "name": "French_bulldog", 
      "prob": 0.01873619481921196
    }, 
    {
      "name": "bull_mastiff", 
      "prob": 0.016469275578856468
    }, 
    {
      "name": "pug", 
      "prob": 0.0009862546576187015
    }
  ]
}



In [15]:
!ls ./photos/hickory.jpeg

./photos/hickory.jpeg
